## Class for Request

In [3]:
import pandas as pd
import datetime
import matplotlib
import matplotlib.pyplot as plt
import math
from matplotlib.patches import Patch
from matplotlib.lines import Line2D

In [26]:
def evaluate_type(df):
    # request type
    index_of_first_sent_packet = df[df['Direction'].str.contains('Sent')].index.min()
    first_request = df.loc[index_of_first_sent_packet]['Opcode']
    if 'Read By Group Type Request' in first_request:
        return "READ BY GROUP TYPE REQUEST"
    elif 'Read By Type Request' in first_request:
        return "READ BY TYPE REQUEST"
    elif 'Write Command' in first_request:
        return "WRITE COMMAND"
    elif 'Exchange MTU Request' in first_request:
        return "EXCHANGE MTU"
    elif 'Find Information Request' in first_request:
        return "FIND INFO"
    elif 'Prepare Write Request' in first_request:
        return "PREPARE WRITE"
    elif 'Read' in first_request:
        return "READ"
    elif 'Write' in first_request:        
        if '52454144' in str(df.loc[index_of_first_sent_packet]['Value']):
            return "SPECIAL WRITE"
        return "WRITE"
    else:
        print("cant resolve: ",first_request)
        return "UNKOWN"
def evaluate_protocol(df):
    index_of_first_sent_packet = df[df['Direction'].str.contains('Sent')].index.min()
    first_request = df.loc[index_of_first_sent_packet]['Protocol']
    return first_request

def evaluate_sender(df):
    index_of_first_sent_packet = df[df['Direction'].str.contains('Sent')].index.min()
    receiver = df.loc[index_of_first_sent_packet]['Destination']
    sender =  df.loc[index_of_first_sent_packet]['Source']
    return sender,receiver
skip_request_dict = dict()
skip_request_dict[("Read Request", "Read Blob Request")] = ["Read Blob Request"]
skip_request_dict[("Prepare Write Request", "Prepare Write Request")] = ["Prepare Write Request","Execute Write Request"]

CONNECTION_INTERVAL_FACTOR = 1.25
DEFAULT_NA_TIMEDIFF_VALUE = datetime.timedelta(0)

In [2]:
class Request:
    request_type = "unknown"
    dataframe = None
    request_dataframe = None
    response_dataframe = None
    
    start_request_index = -1
    end_request_index = -1
    mtu_size = 0
    max_mtu_size = 0
    request_amount = 0
    response_amount = 0
    request_mac = ""
    receiver_mac = ""
    conn_interval = 0
    duration = None
    data = 0
    
    def __init__(self,dataframe,start_index = -1,end_index=-1,request_type="unknown",conn_interval=-1):        
        # setup
        # filter on pakets between exact two devices
        self.request_mac, self.receiver_mac = evaluate_sender(dataframe)
        dataframe = dataframe[(dataframe['Destination'].apply(lambda dest: self.request_mac in dest or self.receiver_mac in dest))]
        dataframe = dataframe[(dataframe['Source'].apply(lambda src: self.request_mac in src or self.receiver_mac in src))]
        
        # filter on protocol
        protocol = evaluate_protocol(dataframe)
        dataframe = dataframe[dataframe['Protocol'].str.contains(protocol)]
        
        if 'HCI_' in protocol:
            return
        if dataframe.empty:
            return
        
        self.request_type = evaluate_type(dataframe)
        
        if(start_index < end_index):
            self.dataframe = dataframe.loc[start_index:end_index]
            self.start_request_index = start_index
            self.end_request_index = end_index
        else:
            self.request_type = evaluate_type(dataframe)
            df_requests = dataframe[(dataframe['Direction'].str.contains('Sent'))]
            if(len(df_requests) > 1):
                # If there are more than one request, evaluate if they are a request sequence
                first_sent = df_requests.index.values[0]
                second_sent = df_requests.index.values[1]
                first_request_type = df_requests.loc[first_sent]['Opcode']
                second_request_type = df_requests.loc[second_sent]['Opcode']
                # combinations of sent packages lead to ending opcodes
                # (Read Request, Read Blob Request) => skip: Read Blob Request
                # (Read Request, Other) => Read Response
                # (Write Request, Other) => Write Response
                # (Prepare Write Request,Execute Write Request) => (Execute Write Response, Other)
                # (Write Command, Other) => (-, Other)
                # (Exchange MTU Request, Other) => (Exchange MTU Response, Other)
                responses = dataframe.loc[first_sent:second_sent]
                if (first_request_type,second_request_type) in skip_request_dict.keys():
                    skip_request_opcode = skip_request_dict[(first_request_type,second_request_type)]
                    other_requests = df_requests.loc[second_sent:]
                    all_other_requests_are_the_same = True
                    # check other requests to be the same
                    for index,next_request in other_requests.iterrows():
                        if not any(opcode in next_request['Opcode'] for opcode in skip_request_opcode):
                            all_other_requests_are_the_same = False
                            second_sent = index
                            #print(next_request['Opcode'], "is not in",skip_request_opcode)
                            break
                    if all_other_requests_are_the_same:
                        #print(" all requests are the same")
                        responses = dataframe.loc[first_sent:]
                    else:
                        responses = dataframe.loc[first_sent:second_sent]
                        
                responses[responses['Direction'].str.contains('Rcvd')]
                if self.request_type == "SPECIAL WRITE":
                    # in this case, the last index of a Handle Value Notification
                    # before the next request
                    responses = responses.loc[:second_sent-1]
                    responses = responses.index.values
                else:
                    responses = responses[responses['Opcode'].str.contains('Response')].index.values
                    
                if (self.request_type == 'WRITE COMMAND'):
                    self.dataframe = dataframe.loc[first_sent]
                    self.start_request_index = first_sent
                    self.end_request_index = first_sent+1
                    self.duration = DEFAULT_NA_TIMEDIFF_VALUE
                    return
                elif len(responses) == 0:
                    print("Could not Create Request because df does not contain Responses for Request",dataframe)
                    return
                last_response = responses[-1]
                self.dataframe = dataframe.loc[first_sent:last_response]
                self.start_request_index = first_sent
                self.end_request_index = last_response
            elif len(df_requests) == 1:
                # If there is only one request => Use That one Request to build Request
                print("only one Request in the dataframe")
                self.start_request_index = df_requests.index.values[0]
                self.end_request_index = dataframe.index.values[-1]
                self.dataframe = dataframe.loc[self.start_request_index:self.end_request_index]
            else:
                pass
        # polish dataframe
        time_reference = self.dataframe.loc[self.start_request_index]['Time.1']
        self.dataframe.insert(2,"Timediff",self.dataframe['Time.1'].apply(lambda t:t-time_reference).values,True)
        self.request_dataframe = self.dataframe[self.dataframe['Direction'].str.contains('Sent')]
        self.response_dataframe = self.dataframe[self.dataframe['Direction'].str.contains('Rcvd')]
        self.dataframe.insert(3,"Paketdiff",self.dataframe['Time.1'].diff().fillna(DEFAULT_NA_TIMEDIFF_VALUE).values,True)

        if self.request_type == "READ":
            self.data = self.response_dataframe['Length'].sum()
        elif self.request_type == "SPECIAL WRITE":
            self.data = self.dataframe[self.dataframe['Opcode'].str.contains("Notification")]['Length'].sum()
        
        # request amount
        self.request_amount = len(self.request_dataframe)
        self.response_amount = len(self.response_dataframe)
        
        # request mac
        self.request_mac = self.dataframe.loc[self.start_request_index]['Source']
        self.receiver_mac = self.dataframe.loc[self.start_request_index]['Destination']
        # duration
        self.duration = self.dataframe.loc[self.end_request_index]['Time.1'] - self.dataframe.loc[self.start_request_index]['Time.1']
        # interval
        if conn_interval > 0:
            self.conn_interval = conn_interval * CONNECTION_INTERVAL_FACTOR
        if self.response_amount > 1:
            self.mtu_size = int(self.response_dataframe['Length'].max())
        
        if self.request_type == 'READ':
            self.data = self.response_dataframe['Length'].apply(lambda x:x-1).sum()
    
    def set_mtu_size(self,mtu_size):
        self.mtu_size = mtu_size
    
    def set_conn_interval(self,interval):
        self.conn_interval = interval * CONNECTION_INTERVAL_FACTOR
    
    def plot(self,absolut_timereference = None,title_extra="",legend=True):
        ymin = 0
        ymax = 2
        if absolut_timereference is None:
            last_time_reference = self.dataframe.loc[self.end_request_index]['Timediff']
        else:
            last_time_reference = absolut_timereference
        fig,ax = plt.subplots(figsize=(6,3))
        plt.title(self.request_type + " " + title_extra)
        plt.ylim([ymin,ymax])
        timediff_min = self.dataframe['Timediff'].apply(lambda t:t.microseconds+t.seconds*1_000_000).min()
        timediff_max = self.dataframe['Timediff'].apply(lambda t:t.microseconds+t.seconds*1_000_000).max()
        puffer = 0
        plt.xlim([timediff_min-puffer,timediff_max+puffer])
        request_times = self.request_dataframe['Timediff'].apply(lambda t:t.microseconds+t.seconds*1_000_000).values
        response_times = self.response_dataframe['Timediff'].apply(lambda t:t.microseconds+t.seconds*1_000_000).values
        print(request_times,response_times)
        if self.conn_interval > 0:
            puffer = self.conn_interval * 1_000 *0.1
            window_edges = [self.conn_interval *1_000 * i for i in range(0,math.ceil(timediff_max/(self.conn_interval*1_000))+1)]
            print(window_edges)
            plt.xlim([0-puffer,max(window_edges)+puffer])

            for i in range(0,len(window_edges)-1):
                if i%2 == 0:
                    rect = matplotlib.patches.Rectangle((window_edges[i],ymin),self.conn_interval*1_000,ymax,linewidth=1,alpha=0.2,color='green')
                else:
                    rect = matplotlib.patches.Rectangle((window_edges[i],ymin),self.conn_interval*1_000,ymax,linewidth=1,alpha=0.2,color='orange')
                ax.add_patch(rect)
        else:
            plt.vlines(request_times,ymin,ymax,linestyles='dotted',color='green',label='Request')
            plt.vlines(response_times,ymin,ymax,linestyles='dotted',color='red',label='Response')

        if (self.request_type == "SPECIAL WRITE"):
            responses = self.dataframe[self.dataframe['Opcode'].str.contains("Response")]
            response_times = responses['Timediff'].apply(lambda t:t.microseconds+t.seconds*1_000_000).values
            plt.vlines(response_times,ymin,ymax,linestyles='dashdot',color='blue',label='Response')
            
            notifications = self.dataframe[self.dataframe['Opcode'].str.contains("Notification")]
            notification_times = notifications['Timediff'].apply(lambda t:t.microseconds+t.seconds*1_000_000).values
            plt.vlines(notification_times,ymin,ymax,linestyles='dotted',color='red',label='Notification')
            
        elif (self.request_type == "READ"):
            read_responses = self.response_dataframe['Timediff'].apply(lambda t:t.microseconds+t.seconds*1_000_000).values
            plt.vlines(read_responses,ymin,ymax,linestyles='dashdot',color='blue',label='Response')
        
        plt.tight_layout()
        plt.xlabel('time since first request in microseconds')
        if legend:
            legend_items = [Patch(facecolor='green',alpha=0.2,linewidth=1,label='M->S'),
                            Patch(facecolor='orange',alpha=0.2,linewidth=1,label='S->M'),
                            Line2D([0],[0],color='blue',label="Response"),
                            Line2D([0],[0],color='red',label="Notification")
                           ]
            plt.legend(handles=legend_items,bbox_to_anchor=(1.05, 1), loc='upper left')
        plt.show()
    
    def as_dict(self):
        representation = dict()
    
        representation["request_type"] = self.request_type 
        representation["start_request_index"] = self.start_request_index 
        representation["end_request_index"] = self.end_request_index 
        representation["mtu_size"] = self.mtu_size 
        representation["max_mtu_size"] = self.max_mtu_size 
        representation["request_amount"] = self.request_amount 
        representation["response_amount"] = self.response_amount 
        representation["request_mac"] = self.request_mac 
        representation["receiver_mac"] = self.receiver_mac 
        representation["conn_interval"] = self.conn_interval 
        representation["duration"] = self.duration 
        representation["data"] = self.data         
        return representation
        
    
    def resolve_ATT_Requests(df,printInfo= False):
        # ATT Requests can only be applied by ATT protocol
        df = df[df['Protocol'].str.contains('ATT')]
        last_rcvd_index = df[df['Direction'].str.contains('Rcvd')].index.max()
        first_sent_index = df[df['Direction'].str.contains('Sent')].index.min()
        latest_request_end_index = first_sent_index
        requests = []
        if printInfo:
            print("first sent index is:",first_sent_index)
            print("last received index is:",last_rcvd_index)
        while latest_request_end_index < last_rcvd_index:
            request = Request(df.loc[latest_request_end_index:])
            if printInfo:
                print("find requests in range [%s:]"%latest_request_end_index)
                print(request.request_type, "created", request.start_request_index,"--",request.end_request_index)
                print("duration:",request.duration,"subrequests:",request.request_amount)
            if request.end_request_index > 0:
                latest_request_end_index = request.end_request_index
                requests.append(request)
            else:
                latest_request_end_index += 1
        return requests
            
    